In [ ]:
####################################
########  必要なライブラリ群 ##########
####################################
# Oracle Database
import oracledb

# 設定読み込み
from config_loader import load_config, get_db_connection_params

In [ ]:
################
#  定数及び変数  # 
################
# .envファイルから環境変数を読み込み
load_config()

# .envファイル内のDB接続パラメータを取得
db_params = get_db_connection_params()

# 接続トライ(接続失敗した場合はここでError)
connection = oracledb.connect(**db_params)
print(connection)

In [ ]:
#####################################
# 1. 新規にテーブルを作成する(初回のみ)  # 
#####################################

# DDL SQL - ソースドキュメント管理テーブル
sql_create_source_documents = """
create table source_documents (
   document_id     raw(16) default sys_guid() primary key,
   filename        varchar2(200),
   filtering       varchar2(200),
   content_type    varchar2(200),
   file_size       number,
   text_length     number,
   registered_date date
)
"""

# DDL SQL - チャンク＋ベクトル管理テーブル
sql_create_chunks = """
create table chunks (
   chunk_id        raw(16) default sys_guid() primary key,
   document_id     raw(16) not null,
   chunk_text      clob,
   embedding       vector,
   registered_date date,
   foreign key (document_id) references source_documents(document_id)
)
"""

# source_documentsテーブルの作成
try:
    with connection.cursor() as cursor:
        cursor.execute(sql_create_source_documents)
        connection.commit()
        print("source_documents テーブルを作成しました")
except oracledb.Error as e:
    if e.args[0].code == 955:
        print("source_documents テーブルは既に存在します")
    else:
        raise

# chunksテーブルの作成
try:
    with connection.cursor() as cursor:
        cursor.execute(sql_create_chunks)
        connection.commit()
        print("chunks テーブルを作成しました")
except oracledb.Error as e:
    if e.args[0].code == 955:
        print("chunks テーブルは既に存在します")
    else:
        raise

In [ ]:
##########################
# 2. 作成したテーブルの確認  # 
##########################

def show_table_structure(table_name):
    cursor = connection.cursor()
    cursor.execute(f"SELECT * FROM {table_name} WHERE 1=0")
    
    print(f"\nテーブル: {table_name}")
    print(f"{'列名':<20} {'型'}")
    print("-" * 40)
    
    for col in cursor.description:
        col_name = col[0]
        col_type = col[1]
        print(f"{col_name:<20} {col_type}")

# 両方のテーブル構造を表示
show_table_structure("source_documents")
show_table_structure("chunks")

In [ ]:
##############################
# (Option) テーブルデータの確認  # 
##############################

def show_table_data(table_name):
    cursor = connection.cursor()
    cursor.execute(f"SELECT * FROM {table_name}")
    
    columns = [col[0] for col in cursor.description]
    rows = cursor.fetchall()
    
    print(f"\nテーブル: {table_name}")
    print(f"レコード数: {len(rows)}")
    print(" | ".join(f"{col:^15}" for col in columns))
    print("-" * (len(columns) * 18))
    
    for row in rows:
        row_data = []
        for col_name, value in zip(columns, row):
            if col_name in ['CHUNK_TEXT', 'EMBEDDING']:
                display_value = str(value)[:15] + "..." if value else "NULL"
            elif value is None:
                display_value = "NULL"
            else:
                display_value = str(value)[:15]
            row_data.append(f"{display_value:^15}")
        print(" | ".join(row_data))

# 両方のテーブルデータを表示
show_table_data("source_documents")
show_table_data("chunks")

In [ ]:
###########################################
# (Option) テーブル内データの削除 (TRUNCATE)  # 
###########################################

cursor = connection.cursor()

# chunksテーブルを先に削除（外部キー制約のため）
#cursor.execute("TRUNCATE TABLE chunks")
#print("chunks テーブルのデータを全削除(TRUNCATE)しました")

# source_documentsテーブルを削除
#cursor.execute("TRUNCATE TABLE source_documents")
#print("source_documents テーブルのデータを全削除(TRUNCATE)しました")

print("※実行する場合はコメントアウトを外してください")

In [ ]:
###################################
# (Option) テーブルの削除 (DROP)  # 
###################################

cursor = connection.cursor()

# chunksテーブルを先に削除（外部キー制約のため）
#cursor.execute("DROP TABLE chunks")
#print("chunks テーブルを削除(DROP)しました")

# source_documentsテーブルを削除
#cursor.execute("DROP TABLE source_documents")
#print("source_documents テーブルを削除(DROP)しました")

print("※実行する場合はコメントアウトを外してください")